In [1]:
# Misc imports
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")
import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
import yaml
from pathlib import Path
# Local imports
from ese.scripts.utils import get_option_product
from ese.experiment.analysis.run_inference import get_cal_stats
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'fillNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'isNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(meth

In [2]:
%%yaml inference_config


experiment:
    exp_root: '?'
    seed: 40
    threshold: 0.5 
    max_ensemble_samples: 5

log:
    root: '?'
    min_fg_pixels: 0 
    log_interval: 10 
    save_preds: False
    log_image_stats: True 
    log_pixel_stats: False 
    gether_inference_stats: True
    track_ensemble_member_scores: False 
    summary_compute_global_metrics: False 

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: '?'
    normalize: False

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3
    square_diff: False 

global_calibration:
    num_prob_bins: 15
    num_classes: 2
    neighborhood_width: 3
    square_diff: False 
    loss_weights: None 

<IPython.core.display.Javascript object>

In [4]:
%%yaml standard_model_cfg 

# For standard datasets
#####################################
data:
    preload: False 

ensemble:
    combine_fn: 'mean'
    combine_quantity: 'probs'
    normalize: False
    num_members: 5

model:
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: '?'
    pretrained_select_metric: "val-dice_score"
    pred_label: '?' 

<IPython.core.display.Javascript object>

In [5]:
exp_name = '06_14_24_DRIVE_ThresholdedGT'
group_dict = {
    "datasets": [
        "DRIVE"
    ],
    "exp_group": exp_name,
    "model_type": "standard",
    "scratch_root": "/storage/vbutoi/scratch/ESE",
    "inf_cfg_root": "/storage/vbutoi/projects/ESE/ese/experiment/configs/inference",
    "base_models_group": "/storage/vbutoi/scratch/ESE/training/06_14_24_DRIVE_ThresholdedGT",
    # "calibrated_models_group": "/storage/vbutoi/scratch/ESE/calibration/06_12_24_WMH_CalibratedTrainSet",
}

## Gather Inference Options.

In [6]:
%%yaml inference_config_opts

calibrator:
    - "Uncalibrated"
    # - "ImageBasedTS"
    # - "LocalTS"
data.split: ("val", "cal")

<IPython.core.display.Javascript object>

In [7]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "defaults"

##################################################
with open(inf_cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)

##################################################
base_cfg = Config(inference_config).update([calibration_cfg, cal_metrics_cfg, standard_model_cfg])

In [8]:
# Get the configs for the different runs.
inf_cfgs = get_ese_inference_configs(
    group_dict=group_dict,
    base_cfg=base_cfg,
    inf_cfg_opts=inference_config_opts
)

WARNING! Calibrator not found, using default calibrator name: Uncalibrated.


In [9]:
len(inf_cfgs)

5

## Running Jobs

In [10]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [11]:
# from ese.experiment.experiment import run_ese_exp

# ###### Run individual jobs
# run_ese_exp(
#     config=inf_cfgs[0], 
#     job_func=get_cal_stats,
#     run_name='debug',
#     show_examples=True,
#     gpu='1',
# ) 

In [12]:
from ese.experiment.experiment import submit_ese_exps 

#### Run Batch Jobs
submit_ese_exps(
    config_list=inf_cfgs,
    job_func=get_cal_stats,
    available_gpus=['0', '1', '2', '3']
)

Submitted job id: 1723451 on gpu: 0.
Submitted job id: 1723561 on gpu: 1.
Submitted job id: 1723694 on gpu: 2.
Submitted job id: 1723827 on gpu: 3.
Submitted job id: 1723961 on gpu: 0.
